# Project

## Libraries

In [ ]:
import requests
import json
import time
import pandas as pd
from tqdm import tqdm
import psycopg2
import yaml

# from pydantic import BaseModel, ValidationError, validator (## for the future)

## Riot Games API

In [ ]:
with open("../config.yml", "r") as ymlfile:
    cfg = yaml.load(ymlfile)

In [ ]:
api_key = cfg['riotgamesapi']['key']
api_header = {
    "X-Riot-Token": api_key
}

In [ ]:
region = 'euw1'

In [ ]:
challenger_url = f'https://{region}.api.riotgames.com/tft/league/v1/challenger?api_key={api_key}'
challenger_response = requests.get(url = challenger_url).json()

In [ ]:
#Writes json response into a dataframe
challenger_response_df = pd.DataFrame(challenger_response['entries'])


In [ ]:
challenger_response_df

In [ ]:
summonerId = challenger_response_df['summonerId'][10]
puuid_url = f'https://euw1.api.riotgames.com/tft/summoner/v1/summoners/{summonerId}'


In [ ]:
puuid_from_summoner = requests.get(puuid_url, headers=api_header).json()

In [ ]:
puuid = puuid_from_summoner['puuid']

In [ ]:
matchid_url = f'https://europe.api.riotgames.com/tft/match/v1/matches/by-puuid/{puuid}/ids?count=20'
matchids = requests.get(matchid_url, headers=api_header).json()

In [ ]:
matchid = matchids[0]

In [ ]:
match_info_url = f'https://europe.api.riotgames.com/tft/match/v1/matches/{matchid}'
match_info = requests.get(match_info_url, headers = api_header).json()

1. Call the api for 1 match
2. Insert into Match
3. Get the Id, you inserted
4. For the 8 players
    - Insert into MatchInfo, with Id from the Match Table
    - Get the id you inserted into the MatchInfo table
    - Insert into MatchTraits
    - Insert into MatchUnits

## Postgresql Database

In [ ]:
params = cfg['postgresql']
# Connect to the PostgreSQL database
conn = psycopg2.connect(**params, port = 5432)
# Create a new cursor - A PostgreSQL database cursor is a read-only pointer that allows a program, regardless of the language used, to access the result set of a query. 
cur = conn.cursor()
# A function that takes in a PostgreSQL query and outputs a pandas database 

In [ ]:
#Loops through rows for df
for i in tqdm(range(0, challenger_response_df.shape[0])):
    summonerId = challenger_response_df["summonerId"][i]
    # do all the columns that are called TBD

    cur.execute(f'''
    INSERT INTO public."Users"(
	    id, "summonerName", "summonerId", "rankedLeague", puuid, leaguepoints, region)
	    VALUES (DEFAULT, 'TBD', 'TBD', 'CHALLENGER', 'Temporary', null, {region})
        WHERE 
        ''')

    conn.commit()

In [ ]:
#Use this cell to delete table data, although ID counter needs to be reset manually in Postgres
cur.execute(f'''
DELETE FROM public."Users"
''')

In [ ]:
conn.commit()

# Try...
#Incorrect, then rollback transaction

Commit or roll-back...

for loop:
take API and put into SQL 
Execute 1 person at a time...
Validation of data type...

psychopg to insert data into database

## Prefect

In [ ]:
import prefect
from prefect import task, Flow

@task
def hello_task():
    logger = prefect.context.get("logger")
    logger.info("Hello world!")

with Flow("hello-flow") as flow:
    hello_task()

flow.run()

In [ ]:
from prefect import task

@task
def extract():
    """Get a list of data"""
    return [1, 2, 3]

@task
def transform(data):
    """Multiply the input by 10"""
    return [i * 10 for i in data]

@task
def load(data):
    """Print the data to indicate it was received"""
    print("Here's your data: {}".format(data))


In [ ]:
from prefect import task, Flow, Parameter


@task(log_stdout=True)
def say_hello(name):
    print("Hello, {}!".format(name))


with Flow("My First Flow") as flow:
    name = Parameter('name')
    say_hello(name)


flow.run(name='world') # "Hello, world!"
flow.run(name='Marvin') # "Hello, Marvin!"